In [ ]:
from dask.distributed import Client
import dask.dataframe as dd

client = Client()
client

In [ ]:
import numpy as np

import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import logging
from dask.distributed import Client

import re
import json

In [ ]:
ddf = dd.read_csv("medium_reviews.csv", blocksize=50e6, usecols=['reviewerID', 'overall','helpful','reviewTime'])

# ddf.head()

In [ ]:


ddf = ddf.assign(reviewYear = lambda x: x.reviewTime.apply(
    lambda x: int(re.search(r"\d{4}", x).group()),
    meta=('reviewTime', 'str')))

def addHelpUnhelp(df):
    helpful_series = df['helpful'].apply(lambda x: np.array(re.findall(r"\d+", x)).astype(int))
    df[['newHelpful', 'totalreviews']] = pd.DataFrame(helpful_series.to_list())
    df['totalreviews'] = df['newHelpful'] + df['totalreviews']
    return df

metadata = {'reviewerID': 'str',
            'helpful': 'str',
            'overall': 'float',
            'reviewTime': 'str',
            'reviewYear': 'float',
            'newHelpful' : 'float',
            'totalreviews': 'float'
            }

ddf = ddf.map_partitions(addHelpUnhelp,
                   #.apply(
    ##lambda x: np.array(re.findall(r"\d+", x)).astype(int),
    meta=metadata
                  )

# ddf.visualize()

In [ ]:
# ddf.compute().head()

In [ ]:
#drop extra columnsTODO

#index??
#empiricaly check partitions
groups = ddf.groupby('reviewerID')

In [ ]:
ret = groups.aggregate(
{
    'reviewTime' : 'count',
    'overall' : 'mean',
    'reviewYear' : 'min',
    'newHelpful' : 'sum',
    'totalreviews' : 'sum',
})

#split_out=4)

ret = ret.reset_index().rename(columns={
                    "reviewTime":"number_products_rated", 
                    "overall":"avg_ratings", 
                    "reviewYear":"reviewing_since", 
                    "newHelpful":"helpful_votes", 
                    "totalreviews":"total_votes"})

# ret.visualize()


In [ ]:
# ret.compute().head()

In [ ]:
# # ret['reviewerID']
# ret.compute().describe()

In [ ]:
# reviews = pd.read_csv("small_reviews.csv")

In [ ]:
# number_products_rated = reviews.groupby('reviewerID').count()['asin']
# avg_ratings = reviews.groupby('reviewerID')['overall'].mean()
# reviewing_since = reviews.groupby('reviewerID')['reviewTime'].min()
# helpful_reviews = reviews.groupby('reviewerID')['helpful'].agg(lambda x: sum([int(i.strip('[]').split(', ')[0]) for i in x]))
# total_reviews = reviews.groupby('reviewerID')['helpful'].agg(lambda x: sum([int(i.strip('[]').split(', ')[0]) + int(i.strip('[]').split(', ')[1]) for i in x]))


In [ ]:

# users = pd.DataFrame(index = set(reviews['reviewerID']))
# users['number_products_rated'] = number_products_rated
# users['avg_ratings'] = avg_ratings
# users['reviewing_since'] = reviewing_since
# users['helpful_reviews'] = helpful_reviews
# users['total_reviews'] = total_reviews
# users.reset_index(inplace=True)
# users = users.rename(columns = {'index':'reviewerID (PRIMARY KEY)'})
# users


In [ ]:
submit = ret.describe().compute().round(2)
with open('results_PA0.json', 'w') as outfile: json.dump(json.loads(submit.to_json()), outfile)